In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import numpy as np
import pandas as pd


import random

random.seed(10)

N=9
nowcast=1

tnoise=0.25

y_input_size=1
z_input_size=1


file="india_segmented.csv"
file="univer_segmented.csv"
file="solartech_segmented.csv"

series=read_csv(file, sep=",", header=0,parse_dates=True)
series=series.values
series


array([[0, '2017-06-13 00:00:00', 0.0, 24.851, 0.0],
       [1, '2017-06-13 00:10:00', 0.0, 24.802, 0.0],
       [2, '2017-06-13 00:20:00', 0.0, 24.758, 0.0],
       ...,
       [17133, '2017-11-21 23:30:00', 0.0, 7.949, 0.0],
       [17134, '2017-11-21 23:40:00', 0.0, 7.861, 0.0],
       [17135, '2017-11-21 23:50:00', 0.0, 7.681, 0.0]], dtype=object)

In [2]:

X1=sensors=np.array(series[:,2])
X2=sensors=np.array(series[:,3])
Y0=sensors=np.array(series[:,4])

print(X1[0])
print(X2[0])
print(Y0[0])

0.0
24.851
0.0


In [3]:
import tensorflow as tf 

X1n=tf.keras.utils.normalize(    X1, axis=-1, order=2)
X1n=np.array(X1n[0])
#X1n=X1


print(X1n)
print(X1n.shape)

X2n=tf.keras.utils.normalize(X2, axis=-1, order=2)
X2n=np.array(X2n[0])
#X2n=X2
print(X2n)
print(X2n.shape)

X0=np.zeros((len(X1n),2))

X0[:,0]=X1n
X0[:,1]=X2n

Y=np.zeros((len(Y0),1))
Y[:,0]=Y0

print(X0)
print(X0.shape)
print(Y.shape)
print(Y)

[0.0 0.0 0.0 ... 0.0 0.0 0.0]
(17136,)
[0.008708218143262458 0.008691047699858978 0.00867562934251708 ...
 0.002785466420699098 0.0027546297060152985 0.0026915546077984362]
(17136,)
[[0.         0.00870822]
 [0.         0.00869105]
 [0.         0.00867563]
 ...
 [0.         0.00278547]
 [0.         0.00275463]
 [0.         0.00269155]]
(17136, 2)
(17136, 1)
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [4]:
X=[]
for i in range(N,len(X0)):
    X.append(X0[i-N:i])
X=np.array(X)    
print(X.shape)

Y=Y[N-nowcast:len(Y)-nowcast]
print(Y.shape)



from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
print(Y[40])
scaler.fit(Y)

Y = scaler.transform(Y)
print(Y[40])

(17127, 9, 2)
(17127, 1)
[98.72755432]
[0.40682649]


In [5]:
print(X[0])
print(Y[0])

[[0.         0.00870822]
 [0.         0.00869105]
 [0.         0.00867563]
 [0.         0.00867878]
 [0.         0.008649  ]
 [0.         0.00857646]
 [0.         0.00851269]
 [0.         0.00850287]
 [0.         0.00851304]]
[0.]


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_val , y_train , y_val = train_test_split(X , Y ,test_size=0.2, random_state=0)

In [7]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, concatenate
from tensorflow.keras.models import Model, Sequential

from keras import backend as K


mae = tf.keras.losses.MeanAbsoluteError()


def make_generator():
    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    latent = Flatten()(x_output)

    noise = Input(shape=(z_input_size,))
    noise_output = Dense(128, activation='relu')(noise)
    noise_output = Dense(64, activation='relu')(noise_output)
    
    flatten = concatenate([latent, noise_output])

    
    concat = Dense(256)(flatten)
    concat = Dense(128)(concat)
    y_fake = Dense(1,activation="sigmoid")(concat)

    model = Model(inputs=[x,noise], outputs=y_fake)
    return model  






def make_discriminator():
    
    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    latent = Flatten()(x_output)

    label = Input(shape=(y_input_size,))
    label_output = Dense(128, activation='relu')(label)
    label_output = Dense(64, activation='relu')(label_output)

    concat = concatenate([latent, label_output])
    concat = Dense(256)(concat)
    concat = Dense(128)(concat)
    validity = Dense(1,activation="sigmoid")(concat)

    model = Model(inputs=[x, label], outputs=validity)
    model.compile(loss=['binary_crossentropy'],optimizer='adam')
    model.summary()
    return model  

x = Input(shape=(N,2))
noise = Input(shape=(z_input_size,))
generator = make_generator()
label_fake = generator([x,noise])


discriminator = make_discriminator()

validity = discriminator([x, label_fake])

discriminator.trainable = False

combined = Model([noise, x], [label_fake,validity])
combined.compile(loss=['binary_crossentropy'],optimizer='adam',metrics=['accuracy'])
#combined.compile(loss=loss_error_gan,optimizer='adam')
combined.summary()

#combined = Sequential()
#combined.add(generator)
#combined.add(lier)
#combined.add(discriminator)
#combined.compile(loss='binary_crossentropy', optimizer='adam')
    


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 9, 2)]       0           []                               
                                                                                                  
 conv1d_3 (Conv1D)              (None, 8, 16)        80          ['input_5[0][0]']                
                                                                                                  
 conv1d_4 (Conv1D)              (None, 6, 32)        1568        ['conv1d_3[0][0]']               
                                                                                                  
 conv1d_5 (Conv1D)              (None, 4, 64)        6208        ['conv1d_4[0][0]']               
                                                                                            

In [8]:
print(x_train.shape)
print(y_train.shape)



#history = generator.fit(x_train,y_train ,
#                    batch_size=64,
#            validation_data=(x_val,y_val),
#            epochs=40)

(13701, 9, 2)
(13701, 1)


In [9]:
#from sklearn.metrics import confusion_matrix
#yhat = generator.predict(x_val)
#yhat=scaler.inverse_transform(yhat).flatten()
#y_val0=scaler.inverse_transform(y_val).flatten()
#print(y_val)
#print(yhat)

#file = open('prediction.csv', 'w')
#file.write('ground truth\tprediction\n')
#for i in range(0,len(y_val)):
#    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
#file.close()

#print("RMSE")
#error=tf.keras.metrics.mean_squared_error(
#    y_val0, yhat
#)
#print(np.sqrt(error))

In [10]:
def generate_true_samples(batch_size):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    x, true_labels = x_train[idx], y_train[idx]
    return x,true_labels

def getNoise():
    return  np.random.uniform(-tnoise, tnoise, (batch_size, z_input_size))



In [11]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

    
n_epochs=60
batch_size=64

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))


file = open('metrics_cgan0.csv', 'w')

for epoch in range(n_epochs):
    print("e:",epoch)
    for batch_idx in range(int(x_train.shape[0] // batch_size)):
        
        
        noise = getNoise()
        x,true_labels = generate_true_samples(batch_size)
        fake_labels = generator.predict([x,noise])

        discriminator.train_on_batch([x, true_labels], valid)
        discriminator.train_on_batch([x, fake_labels+noise], fake)
        discriminator.train_on_batch([x, fake_labels], valid)
        discriminator.train_on_batch([x, true_labels+noise], fake)
        
        x,true_labels = generate_true_samples(batch_size)
        fake_labels = generator.predict([x,noise])
        
        combined.train_on_batch([fake,x], [true_labels,fake])
        combined.train_on_batch([noise,x], [true_labels+noise,valid])
        combined.train_on_batch([fake_labels-true_labels,x], [fake_labels,valid])
        

    print(epoch, " of ", n_epochs)
    
    
    noise = np.zeros((x_val.shape[0], z_input_size))
    yhat = generator.predict([x_val,noise])
   
    yhat0=scaler.inverse_transform(yhat).flatten()
    y_val0=scaler.inverse_transform(y_val).flatten()

    error=tf.keras.metrics.mean_squared_error(
        y_val0, yhat0
    )
    print("error0",np.sqrt(error))    
    file.write(str(np.sqrt(error))+'\t')
    file.write(str(np.sqrt(tf.keras.metrics.mean_squared_error(
        y_val.flatten(), yhat.flatten()
    )))+'\t')
    
    pred_true=discriminator.predict([x_val, y_val])
    print(pred_true[pred_true>0.5])
    acc_true=len(pred_true[pred_true>=0.5])/len(y_val)   
    print("pred_true:",pred_true,acc_true)
    file.write(str(acc_true)+'\t')

    noise = np.random.uniform(-tnoise, tnoise, (x_val.shape[0], z_input_size))
    yhat = generator.predict([x_val,noise])
    pred_false=discriminator.predict([x_val, yhat])
    acc_false=len(pred_false[pred_false<0.5])/len(y_val)   
    print("pred_false:",pred_false,acc_false)
    file.write(str(acc_false)+'\t')

    noise = np.ones((x_val.shape[0], z_input_size))*tnoise
    yhat = generator.predict([x_val,noise])
    pred_false2=discriminator.predict([x_val, yhat+noise/2.0])
    acc_false2=len(pred_false2[pred_false2<0.5])/len(y_val)   
    print("pred_false2:",pred_false2,acc_false2)
    file.write(str(acc_false2)+'\n')

    print(str(np.sqrt(error))+'\t')

    print(str(acc_true)+'\t'+str(np.mean(pred_true)))
    print(str(acc_false)+'\t'+str(np.mean(pred_false)))
    print(str(acc_false2)+'\t'+str(np.mean(pred_false)))

    
file.close()        
        
    #noise = getNoise()
    #x,true_labels = generate_true_samples(batch_size)
    #fake_labels = generator.predict([x,noise])
    #print("true_labels:",true_labels)
    #print("fake:",fake_labels)
    
    #print("g_loss:",g_loss)
    #print("d_loss:",d_loss)

e: 0
2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 0s/step
0  of  60
108/108 [==============================] - 1s 1ms/step
error0 21.463575
108/108 [==============================] - 0s 1ms/step
[0.88431937 0.901798   0.9013926  ... 0.94261026 0.5264326  0.90193444]
pred_true: [[0.88431937]
 [0.901798  ]
 [0.9013926 ]
 ...
 [0.5264326 ]
 [0.90193444]
 [0.48866802]] 0.772913018096906
108/108 [==============================] - 0s 1ms/step
pred_false: [[0.7426072 ]
 [0.79430324]
 [0.9463899 ]
 ...
 [0.32202396]
 [0.9472558 ]
 [0.4058517 ]] 0.2542323409223584
108/108 [==============================] - 0s 953us/step
pred_false2: [[0.30368093]
 [0.28888845]
 [0.287798  ]
 ...
 [0.27888703]
 [0.28918013]
 [0.30574128]] 1.0
21.463575	
0.772913018096906	0.7106904
0.2542323409223584	0.71588415
1.0	0.71588415
e: 1
2/2 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 2ms/step


2/2 [==============================] - 0s 0s/step


...

2/2 [==============================] - 0s 6ms/step


2/2 [==============================] - 0s 0s/step
58  of  60
108/108 [==============================] - 0s 3ms/step
error0 9.28514
108/108 [==============================] - 0s 3ms/step
[0.837288   0.98429203 0.98434883 ... 0.5947851  0.9878109  0.59279865]
pred_true: [[0.837288  ]
 [0.98429203]
 [0.98434883]
 ...
 [0.5947851 ]
 [0.9878109 ]
 [0.59279865]] 0.9906596614127262
108/108 [==============================] - 0s 3ms/step
pred_false: [[0.98081034]
 [0.9836187 ]
 [0.9846902 ]
 ...
 [0.9659157 ]
 [0.9906167 ]
 [0.74668527]] 0.07384705195563339
108/108 [==============================] - 0s 3ms/step
pred_false2: [[2.2726324e-01]
 [2.8078029e-02]
 [2.8124092e-02]
 ...
 [5.2331018e-01]
 [3.5054263e-02]
 [4.7859408e-05]] 0.854932866316404
9.28514	
0.9906596614127262	0.772729
0.07384705195563339	0.7865657
0.854932866316404	0.7865657
e: 59
2/2 [==============================] - 0s 5ms/step


2/2 [==============================] - 0s 17ms/step


2/2 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 0s/step
59  of  60
108/108 [==============================] - 0s 3ms/step
error0 7.1042066
108/108 [==============================] - 0s 3ms/step
[0.9018431  0.99773216 0.9977366  ... 0.64083177 0.99847406 0.60806423]
pred_true: [[0.9018431 ]
 [0.99773216]
 [0.9977366 ]
 ...
 [0.64083177]
 [0.99847406]
 [0.60806423]] 0.9947460595446584
108/108 [==============================] - 0s 3ms/step
pred_false: [[0.99957216]
 [0.9976251 ]
 [0.9812229 ]
 ...
 [0.76701564]
 [0.9980048 ]
 [0.5994565 ]] 0.05808523058960887
108/108 [==============================] - 0s 3ms/step
pred_false2: [[2.3808357e-01]
 [5.5612650e-02]
 [5.5618040e-02]
 ...
 [4.3854406e-01]
 [7.8071505e-02]
 [1.2117104e-07]] 0.8461762988908348
7.1042066	
0.9947460595446584	0.79475236
0.05808523058960887	0.78061205
0.8461762988908348	0.78061205


In [12]:
from sklearn.metrics import confusion_matrix

noise = np.zeros((x_val.shape[0], z_input_size))
yhat = generator.predict([x_val,noise])
    
yhat=scaler.inverse_transform(yhat).flatten()
y_val0=scaler.inverse_transform(y_val).flatten()
print(y_val)
print(yhat)

file = open('prediction20.csv', 'w')
file.write('ground truth\tprediction\n')
for i in range(0,len(y_val)):
    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
file.close()

print("RMSE")
error=tf.keras.metrics.mean_squared_error(
    y_val0, yhat
)
print(np.sqrt(error))

108/108 [==============================] - 0s 3ms/step
[[0.0141186 ]
 [0.        ]
 [0.        ]
 ...
 [0.07686223]
 [0.        ]
 [0.81869012]]
[  4.6227946    0.2351939    0.24211529 ...  11.4234915    0.36212564
 188.85933   ]
RMSE
7.1042066


In [13]:
noise = np.ones((x_val.shape[0], z_input_size))*tnoise
yhat = generator.predict([x_val,noise])
pred_false2=discriminator.predict([x_val, yhat])
acc_false2=len(pred_false2[pred_false2<0.5])/len(y_val)   
print("pred_false2:",pred_false2,acc_false2)


108/108 [==============================] - 0s 3ms/step
pred_false2: [[0.28426376]
 [0.7378714 ]
 [0.7306341 ]
 ...
 [0.44202676]
 [0.69200265]
 [0.60806423]] 0.1640396964389959


In [14]:
    noise = np.ones((x_val.shape[0], z_input_size))*tnoise
    yhat = generator.predict([x_val,noise])
    pred_false2=discriminator.predict([x_val, yhat+tnoise])
    acc_false2=len(pred_false2[pred_false2<0.5])/len(y_val)   
    print("pred_false2:",pred_false2,acc_false2)


108/108 [==============================] - 0s 3ms/step
pred_false2: [[2.3561457e-01]
 [4.1011140e-02]
 [4.1086741e-02]
 ...
 [4.3993732e-01]
 [5.9774566e-02]
 [6.8763246e-17]] 0.8966725043782837
